In [1]:
import argparse
import os
import json
import xml.etree.ElementTree as ET
import numpy as np
import pandas as pd
import logging


In [2]:
data_file = "./BimanualTaxonomyData_Motions/1480_Cut4_c_30_2cm_02.xml"

In [6]:
tree = ET.parse(data_file)
root = tree.getroot()

file_name = data_file.split('/')[-1].split('_')[0]

In [7]:
# Function to parse the XML data file and extract required data
def parse_mmm_data(file_path, dt=0.01):
    tree = ET.parse(file_path)
    root = tree.getroot()

    # Extract Motion section based on the file name
    file_name = file_path.split('/')[-1].split('_')[0]  # Extract "1480" from "1480_Cut1_c_0_05cm_02.xml"
    motion_section = None
    for motion in root.findall(".//Motion"):
        if motion.get('name') == file_name and motion.get('type') == 'object':
            motion_section = motion
            break
    
    if motion_section is None:
        raise ValueError(f"Motion section with name '{file_name}' not found in the XML file.")
    
    # Extract height, mass, and hand length
    height = float(motion_section.find(".//Height").text)
    mass = float(motion_section.find(".//Mass").text)
    hand_length = float(motion_section.find(".//HandLength").text)
    
    # Initialize lists to store parsed data
    timesteps = []
    root_positions = []
    root_rotations = []
    joint_positions = []
    
    # Parse ModelPose data for root positions and rotations
    for sensor in motion_section.findall(".//Sensor[@type='ModelPose']"):
        for measurement in sensor.findall(".//Measurement"):
            timesteps.append(float(measurement.attrib['timestep']))
            root_positions.append(
                list(map(float, measurement.find("RootPosition").text.split()))
            )
            root_rotations.append(
                list(map(float, measurement.find("RootRotation").text.split()))
            )
    
    # Parse Kinematic data for joint positions
    for sensor in motion_section.findall(".//Sensor[@type='Kinematic']"):
        joint_data = {}
        joint_names = [x.get('name') for x in sensor.find('.//Configuration').findall('Joint')]
        joint_data['names'] = joint_names
        joint_data['data'] = []
        
        for measurement in sensor.findall(".//Measurement"):
            joint_data['data'].append(
                [float(measurement.attrib['timestep'])] + list(map(float, measurement.find("JointPosition").text.split()))
            )
        joint_positions.append(joint_data)

    joints_df = []
    for joint_data in joint_positions:
        joints_df.append(pd.DataFrame(joint_data['data'], columns=['timestep'] + joint_data['names']))
        joints_df[-1]['timestep'] = (joints_df[-1].timestep / dt).round() * dt
    
    joints_df = sorted(joints_df, key = lambda x: x.shape[0])
    
    data = pd.merge(joints_df[0], joints_df[1], how='inner', on='timestep').reset_index(drop=True)
    joints_df = pd.merge(data, joints_df[2], on='timestep').reset_index(drop=True)


    root_df = pd.DataFrame(root_positions, columns = ['RootPositionX', 'RootPositionY', 'RootPositionZ'])
    root_df['timestep'] = (np.array(timesteps) / dt).round() * dt 
    root_df[['RootRotationX', 'RootRotationY', 'RootRotationZ']] = root_rotations

    final_data = pd.merge(joints_df, root_df, on='timestep').reset_index(drop=True)

    final_data['Height'] = height
    final_data['Mass'] = mass
    final_data['HandLength'] = hand_length

    return final_data

# Function to convert parsed data into generalized coordinates
def compute_generalized_coordinates(df):
    generalized_coordinates = []

    for _, row in df.iterrows():
        # Concatenate root positions, rotations, and joint positions into q
        q = row['RootPosition'] + row['RootRotation'] + row['JointPositions']
        generalized_coordinates.append(q)

    # Convert to DataFrame for better handling
    q_df = pd.DataFrame(generalized_coordinates)
    q_df.columns = [f"q{i+1}" for i in range(q_df.shape[1])]

    return q_df



In [8]:
# Parse the data
parsed_data = parse_mmm_data(data_file)
parsed_data.to_csv("humanoid_test.csv", index=False)
